<h1 align="center"><font color="yellow">LangChain: Agents 🤖</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

<font color="orange">Os agentes são como "ferramentas" (`Tools`) para `LLMs`. Eles permitem que um LLM acesse a `Google Search`, execute cálculos complexos com `Python` e até mesmo faça consultas `SQL`.

Neste Notebook, exploraremos os `agentes` e como usá-los no `LangChain`.

Começaremos instalando as bibliotecas de pré-requisito que usaremos neste exemplo.</font>

```
!pip install -qU langchain openai google-search-results wikipedia sqlalchemy
```

In [1]:
%pip install -qU langchain openai google-search-results wikipedia sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip show langchain

Name: langchain
Version: 0.0.265
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/venv_LLMs/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


In [4]:
from langchain import OpenAI


Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 

llm = OpenAI(
    openai_api_key=Eddy_API_KEY_OpenAI,
    temperature=0
)

<font color="orange">Como fizemos antes, contaremos nossos tokens em cada chamada.</font>

In [5]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Gastou um total de {cb.total_tokens} Tokens')

    return result

<font color="orange">Com tudo isso configurado, vamos pular para Agentes.</font>


<font color="red">O que é um agente?</font>

<font color="pink">Definição:</font> 

A chave por trás dos `agentes` é dar aos LLM's a possibilidade de usar ferramentas em seu fluxo de trabalho. É aqui que o langchain se afasta da popular implementação do `ChatGPT` e podemos começar a ter uma ideia do que ele nos oferece como construtores. Até agora, cobrimos vários blocos de construção isoladamente. Vamos vê-los ganhar vida.

A definição oficial de agentes é a seguinte:

>> Os agentes usam um LLM para determinar quais ações devem ser executadas e em que ordem. Uma ação pode ser usar uma ferramenta e observar sua saída ou retornar ao usuário.

Nesta edição, abordaremos o que podemos chamar de agentes "genéricos" que realmente são capazes de realizar muitas metatarefas. Existem outros agentes mais específicos que são ajustados para tarefas diferentes (chamados `kits de ferramentas de agente`), mas iremos abordá-los em uma edição futura.


# <font color="red">Criar um Database</font>

<font color="orange">Usaremos os agentes para interagir com um pequeno banco de dados de amostras de ações. Não vamos nos aprofundar nos detalhes porque esta é apenas uma ferramenta fictícia que construiremos para fins ilustrativos. Vamos criá-lo.</font>

In [6]:
from sqlalchemy import MetaData

metadata_obj = MetaData()


from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),    
)



from sqlalchemy import create_engine

engine = create_engine("sqlite:///database_eddy.db")
metadata_obj.create_all(engine)



In [7]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [8]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0], 
    stock_ticker=obs[1], 
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)
        

In [ ]:
for obs in observations:
    insert_obs(obs)
    

In [41]:
# https://python.langchain.com/docs/use_cases/sql
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain


db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)



<font color="orange">Por fim, criaremos uma ferramenta com essa cadeia (`chain`). Para criar uma ferramenta personalizada, precisamos apenas de três entradas:

* `name:` o nome da ferramenta (enviado para o llm como contexto)

* `func:` a função que será aplicada na requisição do `LLM`

* `description:` a descrição da ferramenta, para que ela deve ser usada (enviada para o llm como contexto)</font>

In [42]:
from langchain.agents import Tool

sql_tool = Tool(
    name='Stock DB',
    func=sql_chain.run,
    description="Útil para quando você precisa responder a perguntas sobre ações e seus preços."
)


# <font color="red">Tipos de Agente</font>